In [7]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.ops import nearest_points

social_vul_data = gpd.read_file('Climate_Ready_Boston_Social_Vulnerability.geojson')
transit_data = gpd.read_file('export_mbta_stops.geojson')
air_quality_data = pd.read_csv('air_quality_summary.csv')

mbta_stops_gdf = transit_data[['OBJECTID', 'geometry']]
air_quality_gdf = gpd.GeoDataFrame(air_quality_data, geometry=gpd.points_from_xy(air_quality_data.Longitude, air_quality_data.Latitude))

mbta_stops_within_tracts = []
air_quality_data_for_tracts = []

for index, tract in social_vul_data.iterrows():
    tract_geometry = tract['geometry']
    mbta_stops_within_tract = mbta_stops_gdf[mbta_stops_gdf.intersects(tract_geometry)]
    mbta_stops_count = len(mbta_stops_within_tract)
    mbta_stops_within_tracts.append(mbta_stops_count)

    nearest_sensor = nearest_points(tract_geometry, air_quality_gdf.unary_union)[1]

    nearest_sensor_data = air_quality_gdf[air_quality_gdf['geometry'] == nearest_sensor]

    air_quality_data_for_tracts.append({
        'AQI_OZONE': nearest_sensor_data['AQI_OZONE'].values[0],
        'AQI_PM10': nearest_sensor_data['AQI_PM10'].values[0],
        'AQI_PM2.5': nearest_sensor_data['AQI_PM2.5'].values[0],
        'CategoryName_OZONE': nearest_sensor_data['CategoryName_OZONE'].values[0],
        'CategoryName_PM10': nearest_sensor_data['CategoryName_PM10'].values[0],
        'CategoryName_PM2.5': nearest_sensor_data['CategoryName_PM2.5'].values[0],
    })

social_vul_data['MBTA_Stops_Count'] = mbta_stops_within_tracts
social_vul_data['Air_Quality_Data'] = air_quality_data_for_tracts

m = folium.Map(location=[42.3601, -71.0589], zoom_start=10)

folium.GeoJson(social_vul_data,
               tooltip=folium.GeoJsonTooltip(fields=['FID', 'Name', 'TotChild', 'OlderAdult', 'MedIllnes', 'POC2', 'LEP', 'Low_to_No', 'TotDis', 'MBTA_Stops_Count', 'Air_Quality_Data'])
              ).add_to(m)

m.save('mergedmap.html')
m


In [8]:

social_vul_data['Air_Quality_Tooltip'] = social_vul_data.apply(
    lambda row: f"AQI_OZONE: {row['Air_Quality_Data']['AQI_OZONE']}<br>"
                f"AQI_PM10: {row['Air_Quality_Data']['AQI_PM10']}<br>"
                f"AQI_PM2.5: {row['Air_Quality_Data']['AQI_PM2.5']}<br>"
                f"CategoryName_OZONE: {row['Air_Quality_Data']['CategoryName_OZONE']}<br>"
                f"CategoryName_PM10: {row['Air_Quality_Data']['CategoryName_PM10']}<br>"
                f"CategoryName_PM2.5: {row['Air_Quality_Data']['CategoryName_PM2.5']}<br>"
                if row['Air_Quality_Data'] else "No Air Quality Data",
    axis=1
)

social_vul_data[['FID', 'Name', 'TotChild', 'OlderAdult', 'MedIllnes', 'POC2', 'LEP', 'Low_to_No', 'TotDis', 'MBTA_Stops_Count', 'Air_Quality_Tooltip']].to_csv('tract_data.csv', index=False)
